In [ ]:
import os
import re
import rasterio
from rasterio.enums import Resampling

def create_yearly_tif(folder_path, output_folder):
    # Create a dictionary to store data for each year
    yearly_data = {}
    tif_files = [f for f in os.listdir(folder_path) if f.endswith('.tif')]

    # Iterate through TIF files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".tif"):
            # Extract year from the filename using regular expression
            match = re.match(r"P_CHIRPS.v2.0_mm-month-1_monthly_(\d{4}).\d{2}.\d{2}.tif", filename)
            if match:
                year = match.group(1)

                # Open the TIF file using rasterio
                with rasterio.open(os.path.join(folder_path, filename)) as src:
                    # Read data and resample if needed
                    data = src.read(1, resampling=Resampling.average)

                    # Aggregate data for the year (you might need to adjust this depending on your data format)
                    if year not in yearly_data:
                        print(year)
                        yearly_data[year] = data
                    else:
                        yearly_data[year] += data

    # Create yearly TIF files
    for year, data in yearly_data.items():
        # Define the output file path
        output_path = os.path.join(output_folder, f"P_CHIRPS.v2.0_mm-month-1_yearly_{year}.tif")

        # Get the metadata from one of the input files
        with rasterio.open(os.path.join(folder_path, tif_files[0])) as src:
            profile = src.profile

        # Update metadata with the new output path
#         meta.update(count=1, driver='GTiff', compress='lzw', nodata=None)

        # Write the aggregated data to the new TIF file
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(data, 1)
        print(year)

# Example usage
folder_path = r"/efs/CWA/rs_input_20yrs/All_20/P/Monthly"
output_folder = r"/efs/Uganada/CHIRPS"
create_yearly_tif(folder_path, output_folder)
